In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tensorflow as tf

import numpy as np
import libspn as spn

from deepsm.graphspn.spn_model import SpnModel, mod_compute_graph_up
from deepsm.graphspn.tbm.spn_template import TemplateSpn, NodeTemplateSpn, EdgeRelationTemplateSpn
from deepsm.graphspn.tbm.spn_instance import NodeTemplateInstanceSpn
from deepsm.graphspn.tbm.template import ThreeRelTemplate, SingleRelTemplate, SingleTemplate, RelTemplate
from deepsm.graphspn.tbm.graph_builder import build_graph
from deepsm.util import CategoryManager

# Load a trained view template SPN

In [3]:
spn_path = "/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/EdgeRelationTemplateExperiment/models/RelTemplate_2_Stockholm456.spn"

In [4]:
sess = tf.Session()

In [5]:
template_spn = EdgeRelationTemplateSpn(ThreeRelTemplate, seed=100)
#template_spn.load(spn_path, sess)
SpnModel.print_structure(template_spn._root, sess)
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

Generating SPN structure...
[Using seed 100]
Node (Sum)
    Input(Products1/Product1, None) (Product)
    Input(Products1/Product2, None) (Product)
    Input(Products1/Product3, None) (Product)
    Input(Products1/Product4, None) (Product)
Products1/Product1 (Product)
    Input(Sums1.1/Sum1, [0]) (Sum)
    Input(Sums1.2/Sum1, [0]) (Sum)
Products1/Product2 (Product)
    Input(Sums1.1/Sum2, [0]) (Sum)
    Input(Sums1.2/Sum1, [0]) (Sum)
Products1/Product3 (Product)
    Input(Sums1.1/Sum1, [0]) (Sum)
    Input(Sums1.2/Sum2, [0]) (Sum)
Products1/Product4 (Product)
    Input(Sums1.1/Sum2, [0]) (Sum)
    Input(Sums1.2/Sum2, [0]) (Sum)
Sums1.1/Sum1 (Sum)
    Input(Products2/Product1, None) (Product)
    Input(Products2/Product2, None) (Product)
    Input(Products2/Product3, None) (Product)
    Input(Products2/Product4, None) (Product)
    Input(Products2/Product5, None) (Product)
    Input(Products2/Product6, None) (Product)
    Input(Products2/Product7, None) (Product)
    Input(Products2/Pro

In [6]:
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

Conc_EdgeRelationTemplateSpn_3_1
View_EdgeRelationTemplateSpn_3_1


In [7]:
template_spn.expand()

In [8]:
spn.display_spn_graph(template_spn._root, skip_params=False)

In [11]:
template_spn.init_weights_ops()
sess.run(template_spn._initialize_weights)

Generating weight initialization Ops...


In [12]:
template_spn.init_learning_ops()

Initializing learning Ops...


StructureError: Sums1.2/Sum2 is missing weights

In [11]:
template_spn.template

deepsm.graphspn.tbm.template.ThreeRelTemplate

In [12]:
SpnModel.print_structure(template_spn._root, sess)

Node (Sum)
    Input(Products1/Product1, None) (Product)
    Input(Products1/Product2, None) (Product)
    Input(Products1/Product3, None) (Product)
    Input(Products1/Product4, None) (Product)
Products1/Product1 (Product)
    Input(Sums1.1/Sum1, [0]) (Sum)
    Input(Sums1.2/Sum1, [0]) (Sum)
Products1/Product2 (Product)
    Input(Sums1.1/Sum2, [0]) (Sum)
    Input(Sums1.2/Sum1, [0]) (Sum)
Products1/Product3 (Product)
    Input(Sums1.1/Sum1, [0]) (Sum)
    Input(Sums1.2/Sum2, [0]) (Sum)
Products1/Product4 (Product)
    Input(Sums1.1/Sum2, [0]) (Sum)
    Input(Sums1.2/Sum2, [0]) (Sum)
Sums1.1/Sum1 (Sum)
    Input(Products2/Product1, None) (Product)
    Input(Products2/Product2, None) (Product)
    Input(Products2/Product3, None) (Product)
    Input(Products2/Product4, None) (Product)
    Input(Products2/Product5, None) (Product)
    Input(Products2/Product6, None) (Product)
    Input(Products2/Product7, None) (Product)
    Input(Products2/Product8, None) (Product)
    Input(Products2/Pr

In [13]:
SpnModel.print_weights(template_spn._root, sess)

Node (Sum)


AttributeError: 'NoneType' object has no attribute 'get_value'

In [14]:
template_spn._conc_inputs.set_inputs()
print(template_spn._conc_inputs.inputs)
print(template_spn._num_nodes)
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

()
3
Conc_EdgeRelationTemplateSpn_3_1
View_EdgeRelationTemplateSpn_3_1


In [71]:
catg_inputs = spn.IVs(num_vars=6, num_vals=CategoryManager.NUM_CATEGORIES, name="Catg")
view_inputs = spn.IVs(num_vars=2, num_vals=5, name="View")
conc = spn.Concat(catg_inputs, view_inputs)
copied_tspn_root = mod_compute_graph_up(template_spn._root,
                                        TemplateSpn.dup_fun_up,
                                        tmpl_num_vars=[0, 1],
                                        tmpl_num_vals=[CategoryManager.NUM_CATEGORIES, 5],
                                        graph_num_vars=[catg_inputs.num_vars, view_inputs.num_vars],
                                        conc=conc,
                                        labels=[[],[0]])

In [72]:
SpnModel.print_weights(copied_tspn_root, sess)

Sum (Sum)
[ 0.0035598   0.34087345  0.29326344  0.24100113  0.12130217] [(5,)]


In [73]:
SpnModel.print_structure(copied_tspn_root, sess)

Sum (Sum)
    Input(Product_4, None) (Product)
    Input(Product_3, None) (Product)
    Input(Product_2, None) (Product)
    Input(Product_1, None) (Product)
    Input(Product, None) (Product)
Product_4 (Product)
    Input(Concat_1, [12]) (Concat)
Product_3 (Product)
    Input(Concat_1, [13]) (Concat)
Product_2 (Product)
    Input(Concat_1, [14]) (Concat)
Product_1 (Product)
    Input(Concat_1, [15]) (Concat)
Product (Product)
    Input(Concat_1, [16]) (Concat)
Concat_1 (Concat)
    Input(Catg_1, None) (IVs)
    Input(View_1, None) (IVs)


In [74]:
copied_tspn_root.is_valid()

True